Задание 1**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [108]:
'''
Комментарий к решению:
Исходя из градации, вероятно, предполгается, что рейтинг надо оценивать построчно в _исходных_ данных.
Но т.к. каждый фильм может встречаться несколько раз (например, "История игрушек" имеет 247 оценок от 1 до 5),
то вероятна ситуация, когда один и тот же фильм будет классифицирован и с признаком "низкий рейтинг", и "высокий рейтинг".

Чтобы этого избежать, сначала сделана группировка с подсчетом среднего рейтинга.
В результате получается датафрейм, в котором каждый фильшм встречается только один раз.
После этого к каждому фильму и его среднему рейтингу применяется функция,
расставляющая признаки рейтинга ("высокий", "средний", "низкий")

'''

import pandas as pd

def rate_class(class_):
    return a

# df['class'] = df['rating'].apply[rate_class]

def rate_class(row):
    ''' функция проставляет градацию рейтинга, перебирая датафрейм посторочно'''
    if row['rating'] <= 2:
        return 'низкий рейтинг'
    elif row['rating'] <= 4:
        return 'средний рейтинг'
    else:
        return 'высокий рейтинг'


# создаем датафреймы
df_movies = pd.read_csv('movies.csv')
df_ratings = pd.read_csv('ratings.csv')

# объединяем датафреймы по movieId (left join)
new_df = df_movies.merge(df_ratings, on='movieId', how='left')
# print(new_df.head(20))

# группировка по двум графам (можно было просто по title)
group_df = new_df.groupby(['movieId', 'title'])
# средняя по графе 'rating'
result = group_df.rating.mean().reset_index()
# print(result.head(10))

# вывод результата работы функции в новую графу 
result['class'] = result.apply(rate_class, axis=1)

# вывод части результирующего датафрейма
print(result[31:53])


   movieId                               title    rating            class
0        1                    Toy Story (1995)  3.872470  средний рейтинг
1        2                      Jumanji (1995)  3.401869  средний рейтинг
2        3             Grumpier Old Men (1995)  3.161017  средний рейтинг
3        4            Waiting to Exhale (1995)  2.384615  средний рейтинг
4        5  Father of the Bride Part II (1995)  3.267857  средний рейтинг
5        6                         Heat (1995)  3.884615  средний рейтинг
6        7                      Sabrina (1995)  3.283019  средний рейтинг
7        8                 Tom and Huck (1995)  3.800000  средний рейтинг
8        9                 Sudden Death (1995)  3.150000  средний рейтинг
9       10                    GoldenEye (1995)  3.450820  средний рейтинг


Задание 2  
Используем файл keywords.csv.  

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.  

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:  

```geo_data = {  
        'Центр': ['москва', 'тула', 'ярославль'],  
        'Северо-Запад': ['петербург', 'псков', 'мурманск'],  
        'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']  
        }```  

Результат классификации запишите в отдельный столбец region.

In [42]:
import pandas as pd
geo_data = {  
        'Центр': ['москва', 'тула', 'ярославль'],  
        'Северо-Запад': ['петербург', 'псков', 'мурманск'],  
        'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']  
        }

def geo_classification(row):
    ''' функция ищет слова из списка значений в графе датафрейма,
        перебираемой посторочно'''
    region_name = ''.join([k for k, v in geo_data.items() if v[0] in row['keyword'] or v[1] in row['keyword'] or v[2] in row['keyword']])
    if region_name == '':
        return 'undefined'
    else:
        return region_name


df = pd.read_csv('keywords.csv')
df['region'] = df.apply(geo_classification, axis=1)
# df.to_csv('now_df') # вывод в файл для контроля
print(df[126:129], df[1062:1065], df[1235:1238], sep='\n\n') # вывод фрагментов датафрейма

             keyword   shows     region
126  вк моя страница  979105  undefined
127     авито москва  979292      Центр
128      спортмастер  968154  undefined
                                              keyword   shows          region
1062                                    ким кардашьян  176743       undefined
1063                              фарпост владивосток  176951  Дальний Восток
1064  сетевой город образование волгоградской области  167436       undefined
                   keyword   shows        region
1235              три кота  155975     undefined
1236  банк санкт петербург  174375  Северо-Запад
1237               санлайт  155484     undefined


Задание 3 (бонусное)  

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

P/S/ все файлы находятся в архиве «дополнительные файлы к лекции и дз»

In [ ]:
# задание в процессе решения